In [1]:
from pipeline.preprocessing import * # fix later
from pipeline.dataloader import PhonocardiogramAudioDataset, PhonocardiogramByIDDatasetOnlyResult, PhonocardiogramAugmentationTSV
from tqdm import tqdm
from pipeline.utils import compose_feature_label, audio_random_windowing, energy_band_augmentation_random_win
import sys
import itertools
import csv
from datetime import datetime

In [3]:
from pathlib import Path
from torch.utils.data import DataLoader
import torch
import re

In [4]:
file = Path(".") / "assets" / "the-circor-digiscope-phonocardiogram-dataset-1.0.3"
# Training On CSV data
original_data = pd.read_csv(str(file  / "training_data.csv"))
    
model_df = data_wrangling(original_data)
X_CSV = one_hot_encoding(model_df, [
    'Murmur', 
    'Systolic murmur quality', 
    'Systolic murmur pitch',
    'Systolic murmur grading', 
    'Systolic murmur shape', 
    'Systolic murmur timing',
    'Diastolic murmur quality', 
    'Diastolic murmur pitch',
    'Diastolic murmur grading', 
    'Diastolic murmur shape', 
    'Diastolic murmur timing',
])
y_CSV = model_df['Outcome']

c:\Python Work\Senior Design Local\HeartDiseasePrediction\pipeline\preprocessing.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data["Age"].fillna(
c:\Python Work\Senior Design Local\HeartDiseasePrediction\pipeline\preprocessing.py:27: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data["Outcom

In [5]:
# Training on actual patient audio files
segmentation_table = PhonocardiogramAugmentationTSV(file / "training_data")

def augmentation(data, sr=4000, window_length_hz=200, window_len_sec =5.):
    # This augmentation WILL conflict with new feature of frequency based extraction. ->
    x = data
    # x = energy_band_augmentation_random_win(x, sr=sr, window_hz_length=window_length_hz)
    # x = np.fft.ifft(x).real
        
    x = audio_random_windowing(x, window_len_sec)
    return x

In [6]:
def feature_csv(file):
    match = re.match(r'(\d+)_(AV|TV|MV|PV|Phc)', os.path.basename(file))
    key = int(match.group(1))
    record = X_CSV.loc[original_data["Patient ID"] == key].to_numpy()[0]
    return record

def compose_with_csv(file, audio_extracted_features_label):
    feature, y = audio_extracted_features_label
    csv_feat = feature_csv(file)
    return np.concatenate([feature, csv_feat], axis=0), y

In [7]:
lookup = PhonocardiogramByIDDatasetOnlyResult(str(file / "training_data.csv"))
dset = PhonocardiogramAudioDataset(
    file / "training_data",
    ".wav",
    "*", # Everything
    transform=lambda f: (augmentation(librosa.load(f)[0],4000,300,3.),int(lookup[f])),
    balancing=True,
    csvfile=str(file / "training_data.csv"),
    shuffle=True
)

loader = DataLoader(
    dset, 
    batch_size=1,
    shuffle=True
)
X = []
y = []

for resample in range(BATCHING := 1):
    for i in tqdm(loader): # very slow 
        X_i,y_i = i
        X.append(X_i)
        y.append(y_i)

# Creating 1 large matrix to train with classical models
X = torch.cat(X, dim=0)
y = torch.cat(y, dim=0)

100%|██████████| 3060/3060 [00:52<00:00, 58.45it/s]


In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D,Lambda, AveragePooling1D, MaxPooling1D, Flatten,Reshape, Dense, Dropout, BatchNormalization, concatenate
from tensorflow.keras.optimizers import Adam
def get_cnn_model(input_shape):
    cnn_model = Sequential()

    # Convolutional layer
    cnn_model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
    cnn_model.add(BatchNormalization())
    cnn_model.add(MaxPooling1D(pool_size=2))
    cnn_model.add(Dropout(0.2))

    # Another convolutional layer
    cnn_model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
    cnn_model.add(BatchNormalization())
    cnn_model.add(MaxPooling1D(pool_size=2))
    cnn_model.add(Dropout(0.2))

    # Third convolutional layer
    cnn_model.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
    cnn_model.add(BatchNormalization())
    cnn_model.add(MaxPooling1D(pool_size=2))
    cnn_model.add(Dropout(0.3))

    # Flattening followed by dense layers
    cnn_model.add(Flatten())
    cnn_model.add(Dense(256, activation='relu'))
    cnn_model.add(Dropout(0.5))
    cnn_model.add(Dense(1, activation='sigmoid'))  
    # Compile the model
    optimizer = Adam(learning_rate=0.001)
    cnn_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    return cnn_model

In [11]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import precision_score, recall_score, f1_score
def cnn_train(X,y):
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

    cnn = get_cnn_model((X_train.shape[1],1))
    cnn.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_val, y_val), verbose=1)

    probabilities = cnn.predict(X_test)
    threshold = 0.5
    y_pred = (probabilities >= threshold).astype(int)

    # y_pred = np.round(y_pred).astype(int)  # Convert probabilities to binary labels

    acc = metrics.accuracy_score(y_test, y_pred)
    fpr, tpr, _thresholds = metrics.roc_curve(y_test, y_pred)
    auc = metrics.auc(fpr, tpr)
    f1 = f1_score(y_test, y_pred)

    print(f"Accuracy: {acc}")
    print(f"Auc: {auc}")
    print(f"F1 Score: {f1}")
    acc = round(acc * 100, 2)
    auc = round(auc * 100, 2)
    f1 = round(f1 * 100, 2)
    return acc, auc, f1

In [ ]:
# Training Pipeline
acc, auc, f1 = cnn_train(X,y)